In [1]:
import numpy as np
import json as json

In [2]:
proteins = ['A3D',  'FNfn10',  'Fyn_SH3',   'TNfn3', 'M_FABP', 'A_LBP']

In [3]:
def get_dummy_avgs(methyl_S2_axis, CC_atoms, dummy_avgs_dict=None, dummy_errs_dict=None):
    if dummy_avgs_dict == None and dummy_errs_dict == None:
        dummy_avgs = {'ILE_CB_CG2':[], 'ILE_CG1_CD1':[], 'VAL_CB_CG1':[], 'VAL_CB_CG2':[], 'LEU_CG_CD1':[], 'LEU_CG_CD2':[], 'THR_CB_CG2':[], 'ALA_CA_CB':[], 'MET_CG_CE':[]}
        dummy_errs = {'ILE_CB_CG2':[], 'ILE_CG1_CD1':[], 'VAL_CB_CG1':[], 'VAL_CB_CG2':[], 'LEU_CG_CD1':[], 'LEU_CG_CD2':[], 'THR_CB_CG2':[], 'ALA_CA_CB':[], 'MET_CG_CE':[]}
    elif dummy_avgs_dict == None or dummy_errs_dict == None:
        raise ValueError('dummy_avgs_dict and dummy_errs_dict must be both None or both not None')
    else:
        dummy_avgs = dummy_avgs_dict
        dummy_errs = dummy_errs_dict

    restypes_S2 = 'ILE VAL LEU THR ALA MET'.split()
    for restype in restypes_S2:
        for residue in methyl_S2_axis.keys():
            if restype in residue:
                for i,S2_axis in enumerate(methyl_S2_axis[residue]):
                    if S2_axis[0] == 'NaN':
                        continue
                    atom_types = CC_atoms[restype][i]
                    dummy_avgs[f'{restype}_{atom_types[0]}_{atom_types[1]}'].append(S2_axis[0])
                    dummy_errs[f'{restype}_{atom_types[0]}_{atom_types[1]}'].append(S2_axis[1])    

    return dummy_avgs, dummy_errs

In [4]:
for i,protein in enumerate(proteins):

    with open(f'../data/S2_axis_expt/{protein}.json', 'r') as openfile:
        json_object = json.load(openfile)
    protein_S2_axis_expt = json_object['data']
    CC_atoms = json_object['CC_atoms']

    if i==0:
        dummy_avgs, dummy_errs = get_dummy_avgs(protein_S2_axis_expt, CC_atoms)
    else:
        dummy_avgs, dummy_errs = get_dummy_avgs(protein_S2_axis_expt, CC_atoms, dummy_avgs, dummy_errs)


In [5]:
for methyl_type in dummy_avgs.keys():
    dummy_avgs[methyl_type] = np.average(dummy_avgs[methyl_type])
    dummy_errs[methyl_type] = np.sqrt(np.sum(np.square(dummy_errs[methyl_type]))) / len(dummy_errs[methyl_type])

In [6]:
with open("../data/S2_axis_expt/expt_dummymodel.json", "w") as outfile:
    json.dump({'dummy_avgs':dummy_avgs, 'dummy_errs':dummy_errs}, outfile)